# Fine-tuning BERT for multi-label text classification

## 환경 설정

HuggingFace Transformers, Datasets 라이브러리를 설치합니다.

In [2]:
!pip install -q transformers datasets

In [1]:
import pandas as pd
from datasets import load_dataset, ReadInstruction
from transformers import AutoTokenizer
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

C:\Users\wjsrn\anaconda3\envs\py39t116\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## train test로 파일을 분리합니다.

In [3]:
import pandas as pd
df = pd.read_csv("./data/220712_sentiment_anlysis_result_data.csv", index_col=0)
print(len(df))
df.head(3)

401832


,Date,Title,주가의 날짜,등락률(y),updown,Pos,Neg,Mid,감성지수
0,2021-01-04,코스피 마감 1조 담은 동학개미 덕에 신축년 첫날 2944선 기염,2021-01-05,2.24,1,1,-1,0,1
1,2021-01-04,코스피 사상 첫 2900 돌파,2021-01-05,2.24,1,1,-1,0,1
2,2021-01-04,美 CNBC 주요국 뛰어넘은 韓증시 수익률 올해 랠리 계속 극찬,2021-01-05,2.24,1,1,-1,0,1


In [4]:
df.head()

,Date,Title,주가의 날짜,등락률(y),updown,Pos,Neg,Mid,감성지수
0,2021-01-04,코스피 마감 1조 담은 동학개미 덕에 신축년 첫날 2944선 기염,2021-01-05,2.24,1,1,-1,0,1
1,2021-01-04,코스피 사상 첫 2900 돌파,2021-01-05,2.24,1,1,-1,0,1
2,2021-01-04,美 CNBC 주요국 뛰어넘은 韓증시 수익률 올해 랠리 계속 극찬,2021-01-05,2.24,1,1,-1,0,1
3,2021-01-04,서학개미 유치 전쟁 해외계좌 가입하면 돈 드려요,2021-01-05,2.24,1,1,-1,0,1
4,2021-01-04,특징주 면역항암제 개발 EDGC 보합세 0 11 하락,2021-01-05,2.24,1,1,-1,0,1


In [15]:
import re

' 코스피 마감 1조 담은 동학개미 덕에  신축년 첫날 2944선  기염 '

In [17]:
for i in range(len(df.index)):
    df.iloc[i,1] = re.sub('삼성', ' ', df.iloc[i,1])

In [18]:
train_df = df.sample(n=len(df)-4000, random_state = 123, replace = False)
test_df = df.sample(n=4000, random_state = 123, replace = False)

In [19]:
print(train_df.shape)
print(test_df.shape)

(397832, 9)
(4000, 9)


In [20]:
train_df.to_csv("./data/train_7.csv",encoding='utf-8-sig',index = False)
test_df.to_csv("./data/test_7.csv",encoding='utf-8-sig',index = False)

## 미리 나눠놓은 train.csv, test.csv 파일을 불러옵니다.

In [21]:
### Q1. 데이터 파일을 지정하는 코드를 작성하시오
## 각각 train_data, test_data 의 변수에 지정하는 코드를 작성해 보시오

train_data = './data/train_7.csv'
test_data = './data/test_7.csv'

### datasets 를 이용해 train, test 파일을 로드합니다.

In [22]:
from datasets import load_dataset, ReadInstruction

dataset = load_dataset('csv', data_files={'train': train_data, 'test': test_data})

Using custom data configuration default-a726f066553dc896


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 999.60it/s]


Dataset csv downloaded and prepared to C:\Users\wjsrn\.cache\huggingface\datasets\csv\default-a726f066553dc896\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 124.97it/s]


### train과 test가 로드 된것을 확인합니다.
과제를 수행할 때에는 train / validataion / test 데이터로 분리해서 사용합니다.

In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Date', 'Title', '주가의 날짜', '등락률(y)', 'updown', 'Pos', 'Neg', 'Mid', '감성지수'],
        num_rows: 397832
    })
    test: Dataset({
        features: ['Date', 'Title', '주가의 날짜', '등락률(y)', 'updown', 'Pos', 'Neg', 'Mid', '감성지수'],
        num_rows: 4000
    })
})

train 데이터의 첫 번째 데이터를 확인해봅니다.

In [24]:
example = dataset['train'][500]
example

{'Date': '2021-04-08',
 'Title': ' 뉴스새벽배송 백신 부작용 증세 이슈 부각에 美 증시 혼조',
 '주가의 날짜': '2021-04-09',
 '등락률(y)': 0.91,
 'updown': 0,
 'Pos': 1,
 'Neg': -1,
 'Mid': 0,
 '감성지수': 1}

데이터를 살펴보면 인덱스, 카페명, 리뷰, 라벨로 이뤄져있습니다.

라벨만 따로 리스트에 저장하고 각각의 라벨을 정수에 매핑합니다. 
매핑된 것은 dictionary로 만들어줍니다.

In [27]:
labels = [label for label in dataset['train'].features.keys() if label in ['Pos','Neg','Mid']]
id2label = {idx:label for idx, label in enumerate(labels)}

print(labels)
print(id2label)

['Pos', 'Neg', 'Mid']
{0: 'Pos', 1: 'Neg', 2: 'Mid'}


In [28]:
### Q2. enumerate() 와 dictionary comprehension을 활용하여 label:id 의 쌍을 갖는 dictionary label2id 을 정의하시오.
## id2label 을 지정하는 방식과 유사하니 위를 참조하시면 됩니다.

label2id = {label:idx for idx, label in enumerate(labels)}
print(label2id)

{'Pos': 0, 'Neg': 1, 'Mid': 2}


## Preprocess data

말씀드렸듯이 BERT는 텍스트가 아닌 `input_ids`를 입력으로 받습니다. 이를 위해 우선 훈련된 토크나이저를 활용해 텍스트를 토큰화합니다. 이번 예시에서는 `AutoTokenizer`를 활용해 토큰화를 진행합니다.

In [29]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base") # KlueBERT의 tokenizer를 활용합니다.

def preprocess_data(examples):
    # 배치화된 텍스트를 받습니다.
    text = examples["Title"]
    # 인코딩 합니다.
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=100)
    # 라벨을 배치로 만들어줍니다.
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # numpy array로 만들기 위해 0 매트릭스를 만들어줍니다.
    labels_matrix = np.zeros((len(text), len(labels)))
    # 채웁니다.
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]
        encoding["labels"] = labels_matrix.tolist()
        return encoding

인코딩!

In [30]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Parameter 'function'=<function preprocess_data at 0x00000249689500D0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 4/4 [00:00<00:00, 19.13ba/s]


결과 확인

In [31]:
example = encoded_dataset['train'][2]
print(example.keys())
print(example)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
{'input_ids': [2, 14097, 2008, 7088, 27124, 15053, 6306, 9811, 12881, 24, 2377, 2015, 4158, 21, 7815, 2448, 2030, 4530, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [32]:
tokenizer.decode(example['input_ids'])

'[CLS] Northrop Grumman 주당 4분기 수익 1 97달러 보고 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [33]:
example['labels']

[1.0, 0.0, 0.0]

인코딩된 데이터를 파이토치의 포맷에 맞게 변경합니다.
[참조](https://huggingface.co/docs/datasets/v2.2.1/en/package_reference/main_classes#datasets.Dataset.set_format)

In [34]:
encoded_dataset.set_format("torch")

## 모델 정의

모델을 정의하기 위해 `transformers`에서 제공하는 `AutoModelForSequenceClassification` 모듈을 import합니다. pretrained된 모델의 weight를 로드하면 multi-label-classification을 위한 layer가 random한 weight로 생성됩니다.

In [35]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

## 모델 훈련

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 
허깅페이스에서 제공하는 Trainer API를 사용해 훈련을 시킵니다. 이를 위해 `TrainingArguments`와 `Trainer`를 정의해줘야 합니다.

* `TrainingArguments` : [문서](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments)
* `상세` : [문서](https://huggingface.co/docs/transformers/v4.20.0/en/main_classes/trainer#transformers.TrainingArguments)
* `Trainer` : [참조](https://huggingface.co/transformers/main_classes/trainer.html#id1).

In [37]:
batch_size = 8
metric_name = "f1"
epochs = 5

In [38]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"model",
    evaluation_strategy = "epoch",
    # evaluation_strategy="steps",
    # eval_steps=len(dataset['train'])/epochs,
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True
)

훈련을 위해서는 `compute_metrics` 함수를 정의해야합니다.

In [39]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # 먼저 예측값에 sigmoid를 적용합니다. shape은 다음과 같습니다. (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # 그 다음, threshold를 이용해 정수값으로 변경합니다.
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # 마지막으로 metric을 계산합니다.
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # dictionary를 리턴합니다.
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [40]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [41]:
encoded_dataset['train']['input_ids'][0]

tensor([    2,  1748,  2181,  2583,  6324, 28979,  3604, 16313,  3646,  4218,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [42]:
encoded_dataset['train']['input_ids'][0].type()

'torch.LongTensor'

In [43]:
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.8385, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.1520,  0.8576, -0.0181]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

훈련 시작!

In [44]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [45]:
trainer.train()

C:\Users\wjsrn\anaconda3\envs\py39t116\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 397832
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 248645


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.000000,0.000000,1.000000,1.000000,1.000000
2,0.000000,0.000000,1.000000,1.000000,1.000000
3,0.000000,0.000000,1.000000,1.000000,1.000000
4,0.000000,0.000000,1.000000,1.000000,1.000000
5,0.000000,0.000000,1.000000,1.000000,1.000000


***** Running Evaluation *****
  Num examples = 4000
  Batch size = 8
Saving model checkpoint to model\checkpoint-49729
Configuration saved in model\checkpoint-49729\config.json
Model weights saved in model\checkpoint-49729\pytorch_model.bin
tokenizer config file saved in model\checkpoint-49729\tokenizer_config.json
Special tokens file saved in model\checkpoint-49729\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 8
Saving model checkpoint to model\checkpoint-99458
Configuration saved in model\checkpoint-99458\config.json
Model weights saved in model\checkpoint-99458\pytorch_model.bin
tokenizer config file saved in model\checkpoint-99458\tokenizer_config.json
Special tokens file saved in model\checkpoint-99458\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 8
Saving model checkpoint to model\checkpoint-149187
Configuration saved in model\checkpoint-149187\config.json
Model weights saved in model\che

TrainOutput(global_step=248645, training_loss=2.5569061728406148e-05, metrics={'train_runtime': 38821.3254, 'train_samples_per_second': 51.239, 'train_steps_per_second': 6.405, 'total_flos': 1.02221618360904e+17, 'train_loss': 2.5569061728406148e-05, 'epoch': 5.0})

## Evaluate

훈련 후에는 validation data를 이용해 모델을 평가합니다.

In [46]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 4000
  Batch size = 8


{'eval_loss': 0.0,
 'eval_f1': 1.0,
 'eval_roc_auc': 1.0,
 'eval_accuracy': 1.0,
 'eval_runtime': 21.9455,
 'eval_samples_per_second': 182.27,
 'eval_steps_per_second': 22.784,
 'epoch': 5.0}

## Save the best model

In [47]:
trainer.save_model("./model/checkpoint-best-model7")

Saving model checkpoint to ./model/checkpoint-best-model7
Configuration saved in ./model/checkpoint-best-model7\config.json
Model weights saved in ./model/checkpoint-best-model7\pytorch_model.bin
tokenizer config file saved in ./model/checkpoint-best-model7\tokenizer_config.json
Special tokens file saved in ./model/checkpoint-best-model7\special_tokens_map.json


## Inference

모델에 새로운 문장을 넣어 잘 작동하는지 확인합니다. 

In [53]:
text = "적자 전환"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}   # input으로 들어가는 각각 tensor들이 GPU를 사용하기 위한 코드

outputs = trainer.model(**encoding)   # (**encoding) input으로 encoding된 숫자값이 들어가도록 함

모델의 결과값으로 나오는 logits의 shape은 (batch_size, num_labels) 입니다. 우리는 하나의 문장만 모델에 입력하기 때문에 batch_size가 1이 되어야 합니다.

In [54]:
logits = outputs.logits # labels에 대해 리뷰를 기준으로 모델이 뱉은 각각의 숫자값 (실수)-> logits이라고 함
logits.shape

torch.Size([1, 3])

In [55]:
logits

tensor([[ 17.7434, -23.1236, -23.1257]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

우리가 미리 설정해둔 라벨을 얻기 위해서는 먼저 sigmoid를 적용해야합니다. sigmoid를 적용하면 모든 점수가 0과 1사이의 실수로 변경됩니다. 이를 확률로 계산할 수 있습니다. 

그 후에는 미리 설정해둔 threshold를 넘는 점수에 대해 1로 변경해준다음 앞에서 만들었던 사전을 이용해 텍스트로 변경해줍니다.

In [56]:
# apply sigmoid + threshold   # 확률값 중에 뭐를 기준으로 해서 결과를 뱉어 낼거냐. threshold가 높을수록 아주 높은 값만을 라벨링하게 됨
sigmoid = torch.nn.Sigmoid()   #  sigmoid 함수 호출
probs = sigmoid(logits.squeeze().cpu())  # 각각에 대해 확률값으로 바꿔주는 것 
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.1)] = 1  # threshold 값 : 0.1 --> probs이 0.1 이상인 값들만 1로 바꾸고, 나머지는 0으로 남김  -->  1로 해당하는 값이 최종 라벨링된 값
                                                                            # threshold 값 정하는 기준 : 경험적 or ROC 커브(threshold를 계속 바꿔가면서 그래프에 점을 찍어서 curve를 그린것, FPR가 낮고-TPR가 높은 것에 가까운 점을 갖는(AUC를 가장 높게 만드는) threshold를 찾는것)

### Q3. [optional] dictionary id2label 을 활용하여 현재 예문에 대해서 모델이 inference한 label 값을 출력해 보시오.
## predictions에서 1이 된 라벨의 id를 구한 뒤, id2label 에서 그 라벨을 구하면 됩니다.

predicted_labels = [id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]
print(predicted_labels)

['Pos']


In [57]:
probs

tensor([1.0000e+00, 9.0686e-11, 9.0499e-11], grad_fn=<SigmoidBackward0>)

In [59]:
# (이론) 모델의 Loss function 설명

metric: 훈련이 잘 이루어지고 있는지 확인할 수 있는 척도
    eg. 정확도(accuracy), f1 score 등

loss function: 모델을 최적하화기 위해 정의된 함수. optimizer에 의해 최적화(감소)되는 대상(gradient의 대상)
    eg. cross-entropy, Mean-Square-Error, ... BCEWithLigitsLoss? 등
